In [2]:
import sys
sys.path.append('../')
# 这里是为了能偶访问到非当前文件夹中的包

https://docs.trychroma.com/getting-started

https://zhuanlan.zhihu.com/p/658217843


In [7]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
# from langchain.document_loaders import TextLoader#这个有一点问题
from langchain.document_loaders.unstructured import UnstructuredFileLoader
import sentence_transformers
import torch
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from config import *
from typing import List
import re

In [4]:
class ChineseTextSplitter(CharacterTextSplitter):
    def __init__(self, pdf: bool = False, **kwargs):
        super().__init__(**kwargs)
        self.pdf = pdf

    def split_text(self, text: str) -> List[str]:
        if self.pdf:
            text = re.sub(r"\n{3,}", "\n", text)
            text = re.sub('\s', ' ', text)
            text = text.replace("\n\n", "")
        sent_sep_pattern = re.compile(
            '([﹒﹔﹖﹗．。！？]["’”」』]{0,2}|(?=["‘“「『]{1,2}|$))') 
        sent_list = []
        for ele in sent_sep_pattern.split(text):
            if sent_sep_pattern.match(ele) and sent_list:
                sent_list[-1] += ele
            elif ele:
                sent_list.append(ele)
        return sent_list

In [8]:
filepath="1.pdf"
loader = UnstructuredFileLoader(filepath)
textsplitter = ChineseTextSplitter(pdf=True)
docs = loader.load_and_split(textsplitter)

In [9]:
docs

[Document(page_content='未来预测的一个关键挑战是高度的不确定性，这在很大程度上是由于不知道其他代理的意图和潜在  特征。', metadata={'source': '1.pdf'}),
 Document(page_content='例如，车辆通常具有多模态的未来分布：它可以转弯、直行、减速、加速等。', metadata={'source': '1.pdf'}),
 Document(page_content='根据其他场景元  素，它可以通过、让行、改变车道或驶入车道。', metadata={'source': '1.pdf'}),
 Document(page_content='在过去的几年里，这一挑战引起了很多人的兴趣。', metadata={'source': '1.pdf'}),
 Document(page_content='对高  度多模态进行建模的一种方法是采用可以从中抽取样本的灵活隐式分布——条件变分自动编码器(CVAEs)  [1]、生成对抗网络(GANs)[2]、和单步策略推出方法[3]。', metadata={'source': '1.pdf'}),
 Document(page_content='尽管它们的表现具有竞争力，但使用隐变量来  建模意图会阻止它们被解释，并且通常需要测试时间抽样来评估概率查询（例如，“代理向左转的可能性  有多大？”', metadata={'source': '1.pdf'}),
 Document(page_content='）。', metadata={'source': '1.pdf'}),
 Document(page_content='此外，在大型机器学习社区[4]中，特别是自动驾驶汽车中，已经有相当大的努力来解决这  类模型中的模式崩溃问题[5,6]。', metadata={'source': '1.pdf'}),
 Document(page_content='  为了解决这些限制，我们观察到，对于我们的任务（例如车辆和行人轨迹预测），在较长的未来中  的不确定性主要可以通过对代理可能目标的预测来捕获。', metadata={'source': '1.pdf'}),
 Document(page_content='这些目标不仅基于可解释的物理实体（例如位  置

In [10]:
docs[0].metadata

{'source': '1.pdf'}

In [11]:
docs[0].page_content

'未来预测的一个关键挑战是高度的不确定性，这在很大程度上是由于不知道其他代理的意图和潜在  特征。'

In [12]:
embedding_model_dict = embedding_model_dict
llm_model_dict = llm_model_dict
EMBEDDING_DEVICE = EMBEDDING_DEVICE
LLM_DEVICE = LLM_DEVICE
num_gpus = num_gpus#GPU数量
large_language_model = init_llm
embedding_model=init_embedding_model

In [13]:
model = HuggingFaceEmbeddings(model_name=embedding_model_dict[embedding_model], )
print('第一步加载成功')
model.client = sentence_transformers.SentenceTransformer(
            model.model_name,
            device=EMBEDDING_DEVICE,
            cache_folder=os.path.join(MODEL_CACHE_PATH,model.model_name))
print('embedding模型加载成功')
# 这里相当于是对client属性进行赋值，尽管在__init__huggingface中已经赋值了，但是没全
'''
        self.client = sentence_transformers.SentenceTransformer(
            self.model_name, cache_folder=self.cache_folder, **self.model_kwargs
        )

'''
model.model_name

第一步加载成功
embedding模型加载成功


'F:/大模型源码/embedding/text2vec-base-chinese_old'

In [14]:

db = Chroma.from_documents(docs, model,persist_directory="../langchain_chromadb/vector_store/chroma_1")


In [15]:
db._collection.count()

22

In [11]:
# import chromadb
# from chromadb import Documents, EmbeddingFunction, Embeddings
# chroma_client = chromadb.Client()

# # 这里创建数据不用langchain
# # 这里是对一个一个向量数据库
# class MyEmbeddingFunction(EmbeddingFunction):
#     def __call__(self, texts: Documents) -> Embeddings:
#         embeddings = [model.embed_query(x.page_content) for x in texts]
#         return embeddings

In [12]:
# # create the open-source embedding function
# embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma


In [ ]:
# ids=[f'id{i+1}' for i in range(len(docs))]
# ids

In [ ]:
# # collection 需要确定id
# collection.add(
#     documents=docs,
#     # metadata=metadata,
#     ids=ids
# )